In [1]:
import sys
import os
import time
import pandas as pd 
import numpy as np
from collections import Counter
import datetime
sys.path.append('/Users/yantingting/PycharmProjects/modeling/')
import database_conncet as data_qry
import plot_tools as pt
import data_processing as dp
import plot_tools as pl

In [2]:
file_path = '/Users/yantingting/Seafile/风控/模型/13 菲律宾/线上模型复盘/老客户/'

# 老用户分层

In [170]:
query1 = '''
select t1.order_no as order_key, t1.customer_id as customer_key,t1.customer_phone_num as phone_key,t1.apply_time,
t1.product_name,t1.order_type,t1.cnt_apply,
t2.*
from 
(select order_no,customer_phone_num, customer_id,apply_time,product_name,order_type,
row_number()over(partition by customer_id order by apply_time asc) - 1 as cnt_apply
from  dw_flb_approval_phl_approval_prod_approval_applications)t1
left join 
(select order_no,phone_number,customer_id,loan_days,start_date,due_date,actual_paid_off_date,
repay_status,business_type,extend_period,late_days
from dw_flb_hx_phl_core_prod_core_repayment_schedules
where product_name != 'CashNiJuan') t2
on t1.customer_phone_num  = t2.phone_number
where t1.product_name != 'CashNiJuan' 
and t1.order_type like 'Qua%'
and date(t1.apply_time) >='2019-04-01'
and date(t1.apply_time) <='2019-12-23'
and (date(t1.apply_time)> t2.start_date or t2.start_date is null)
'''

In [171]:
df1 = data_qry.database_gn(query1)
print(1)
df1.info()
df1.head()
df1.to_csv(os.path.join(file_path, 'df.csv'), index=False)
remain_cols1 = ['order_key','apply_time','product_name','order_type']
df1[remain_cols1].drop_duplicates().to_csv(file_path + 'my_info.csv', index = False)

Successful connection
1
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75413 entries, 0 to 75412
Data columns (total 18 columns):
order_key               75413 non-null object
customer_key            75413 non-null int64
phone_key               75413 non-null object
apply_time              75413 non-null datetime64[ns]
product_name            75413 non-null object
order_type              75413 non-null object
cnt_apply               75413 non-null int64
order_no                75413 non-null object
phone_number            75413 non-null object
customer_id             75413 non-null int64
loan_days               75413 non-null int64
start_date              75413 non-null object
due_date                75413 non-null object
actual_paid_off_date    75413 non-null object
repay_status            75413 non-null object
business_type           75413 non-null object
extend_period           75413 non-null int64
late_days               75413 non-null int64
dtypes: datetime64[ns](1), int64(6), 

,order_key,customer_key,phone_key,apply_time,product_name,order_type,cnt_apply,order_no,phone_number,customer_id,loan_days,start_date,due_date,actual_paid_off_date,repay_status,business_type,extend_period,late_days
0,P2g201906250630,17,9672308678,2019-06-25 17:31:53,Peso2Go,QualityApplicationOrderClassB,1,20181220175119100077,9672308678,17,7,2018-12-21,2018-12-27,2018-12-23,ExtensionPaidOff,EXTENSION_REPAYMENT,0,0
1,P2g201906250630,17,9672308678,2019-06-25 17:31:53,Peso2Go,QualityApplicationOrderClassB,1,20181220175119100077,9672308678,17,7,2018-12-28,2019-01-03,2018-12-24,ExtensionPaidOff,EXTENSION_REPAYMENT,1,0
2,P2g201906250630,17,9672308678,2019-06-25 17:31:53,Peso2Go,QualityApplicationOrderClassB,1,20181220175119100077,9672308678,17,7,2018-12-16,2018-12-22,2018-12-26,OverduePaidOff,EXTENSION_REPAYMENT,2,4
3,P2g201907110205,17,9672308678,2019-07-11 10:25:13,Peso2Go,QualityApplicationOrderClassC,2,20181220175119100077,9672308678,17,7,2018-12-21,2018-12-27,2018-12-23,ExtensionPaidOff,EXTENSION_REPAYMENT,0,0
4,P2g201907110205,17,9672308678,2019-07-11 10:25:13,Peso2Go,QualityApplicationOrderClassC,2,20181220175119100077,9672308678,17,7,2018-12-28,2019-01-03,2018-12-24,ExtensionPaidOff,EXTENSION_REPAYMENT,1,0


KeyError: 'remain_cols1'

In [40]:
df1 = pd.read_csv(os.path.join(file_path, 'df.csv'))
df1['apply_repay_day'] = df1.apply(lambda x: (pd.to_datetime(x['apply_time']) - pd.to_datetime(x['actual_paid_off_date'])).days if x['actual_paid_off_date'] != '1970-01-01' else
                                             -9999,  axis =1)

df1['repay_start_day'] = df1.apply(lambda x: (pd.to_datetime(x['actual_paid_off_date']) - pd.to_datetime(x['start_date'])).days if x['actual_paid_off_date'] != '1970-01-01' else
                                             -9999,  axis =1)
df1.head()

,order_key,customer_key,phone_key,apply_time,product_name,order_type,cnt_apply,order_no,phone_number,customer_id,loan_days,start_date,due_date,actual_paid_off_date,repay_status,business_type,extend_period,late_days,apply_repay_day,repay_start_day
0,P2g201906250630,17,9672308678,2019-06-25 17:31:53,Peso2Go,QualityApplicationOrderClassB,1,20181220175119100077,9672308678,17,7,2018-12-21,2018-12-27,2018-12-23,ExtensionPaidOff,EXTENSION_REPAYMENT,0,0,184,2
1,P2g201906250630,17,9672308678,2019-06-25 17:31:53,Peso2Go,QualityApplicationOrderClassB,1,20181220175119100077,9672308678,17,7,2018-12-28,2019-01-03,2018-12-24,ExtensionPaidOff,EXTENSION_REPAYMENT,1,0,183,-4
2,P2g201906250630,17,9672308678,2019-06-25 17:31:53,Peso2Go,QualityApplicationOrderClassB,1,20181220175119100077,9672308678,17,7,2018-12-16,2018-12-22,2018-12-26,OverduePaidOff,EXTENSION_REPAYMENT,2,4,181,10
3,P2g201907110205,17,9672308678,2019-07-11 10:25:13,Peso2Go,QualityApplicationOrderClassC,2,20181220175119100077,9672308678,17,7,2018-12-21,2018-12-27,2018-12-23,ExtensionPaidOff,EXTENSION_REPAYMENT,0,0,200,2
4,P2g201907110205,17,9672308678,2019-07-11 10:25:13,Peso2Go,QualityApplicationOrderClassC,2,20181220175119100077,9672308678,17,7,2018-12-28,2019-01-03,2018-12-24,ExtensionPaidOff,EXTENSION_REPAYMENT,1,0,199,-4


In [180]:
# 
my_group1 = df1.groupby(['order_key'])
frame1 = pd.DataFrame()
frame1['申请笔数'] = my_group1['cnt_apply'].min()
frame1['放款笔数'] = my_group1['order_no'].agg(pd.Series.nunique)
frame1['失败笔数'] = frame1['申请笔数'] - frame1['放款笔数']
frame1['第一次还款距今天数'] = my_group1['apply_repay_day'].max()
frame1['最近一次还款距今天数'] = my_group1['apply_repay_day'].min()
frame1['累计逾期天数'] = my_group1['late_days'].sum()
frame1['最大逾期天数'] = my_group1['late_days'].max()
frame1['最小逾期天数'] = my_group1['late_days'].min()
frame1['最大展期次数'] = my_group1['extend_period'].max()
frame1['最大结清天数(展期算新的订单)'] = my_group1['repay_start_day'].max()
# frame1.head()
# frame1.shape
# 逾期
df11 = df1[df1['late_days']>0]
my_group11 = df11.groupby(['order_key'])
frame11 = pd.DataFrame()
frame11['逾期订单数']  = my_group11['order_no'].agg(pd.Series.nunique)
frame11['逾期次数(展期算新的订单)']  = my_group11['due_date'].agg(pd.Series.nunique)
# frame11.head()

# 展期
df12 = df1[df1['extend_period']>0]
my_group12 = df12.groupby(['order_key'])
frame12 = pd.DataFrame()
frame12['展期订单数'] = my_group12['order_no'].agg(pd.Series.nunique) 
# frame12.head()

df13 = df1.groupby(['order_key', 'order_no'])['extend_period'].max().to_frame()
my_group13 = df13.groupby(['order_key'])
frame13 = pd.DataFrame()
frame13['总展期次数'] = my_group13['extend_period'].sum()
# frame13.head()

df14 = df1.groupby(['order_key', 'order_no']).agg({'start_date':'min', 'actual_paid_off_date': 'max'})
df14['paid_loan_day'] = df14.apply(lambda x: (pd.to_datetime(x['actual_paid_off_date']) - pd.to_datetime(x['start_date'])).days if x['actual_paid_off_date'] != '1970-01-01' else
                                             -9999,  axis =1)
my_group14 = df14.groupby(['order_key'])
frame14 = pd.DataFrame()
frame14['单笔最小结清天数'] = my_group14['paid_loan_day'].min()
frame14['单笔最大结清天数'] = my_group14['paid_loan_day'].max()
# frame14.head()

my_var1 = pd.merge(frame1 ,frame11, left_index= True, right_index = True, how='left')
my_var11 = pd.merge(my_var1 ,frame12, left_index= True, right_index = True, how='left')
my_var12 = pd.merge(my_var11 ,frame13, left_index= True, right_index = True, how='left')
my_var13 = pd.merge(my_var12 ,frame14, left_index= True, right_index = True, how='left')
my_var13.head()
my_var13.to_csv(file_path + 'my_var1.csv')


,申请笔数,放款笔数,失败笔数,第一次放款距今天数,最近一次放款距今天数,第一次还款距今天数,最近一次还款距今天数,累计逾期天数,最大逾期天数,最小逾期天数,最大展期次数,最大结清天数(展期算新的订单),逾期订单数,逾期次数(展期算新的订单),展期订单数,总展期次数,单笔最小结清天数,单笔最大结清天数
order_key,,,,,,,,,,,,,,,,,,
Jp201910280149,1,1,0,34,34,20,20,0,0,0,0,14,NaN,NaN,NaN,0,14,14
Jp201910280242,1,1,0,34,34,26,26,0,0,0,0,8,NaN,NaN,NaN,0,8,8
Jp201910300455,1,1,0,36,36,23,23,0,0,0,0,13,NaN,NaN,NaN,0,13,13
Jp201911050356,1,1,0,42,42,29,29,0,0,0,0,13,NaN,NaN,NaN,0,13,13
Jp201911070644,1,1,0,44,44,30,30,1,1,1,0,14,1.0,1.0,NaN,0,14,14


In [181]:
query2 = '''
select order_no,
case when extend_period = 0 then late_days else null end as late_days1,
min(start_date) as effective_date,
max(late_days) as max_latedays,
sum(late_days) as sum_latedays
from dw_flb_hx_phl_core_prod_core_repayment_schedules
where product_name != 'CashNiJuan' and order_type like 'Qua%'
and due_date<= '2019-12-15'
group by order_no,
case when extend_period = 0 then  late_days else null end 
'''

In [182]:
df2 = data_qry.database_gn(query2)
df2 = df2[~df2['late_days1'].isnull()]
df2.info()
df2.head()
df2.to_csv(os.path.join(file_path, 'df_flag.csv'), index=False)

Successful connection
<class 'pandas.core.frame.DataFrame'>
Int64Index: 28483 entries, 0 to 34166
Data columns (total 5 columns):
order_no          28483 non-null object
late_days1        28483 non-null float64
effective_date    28483 non-null object
max_latedays      28483 non-null int64
sum_latedays      28483 non-null int64
dtypes: float64(1), int64(2), object(2)
memory usage: 1.3+ MB


,order_no,late_days1,effective_date,max_latedays,sum_latedays
0,20181226112412528091,7.0,2018-12-26,7,7
1,20181228143537773076,0.0,2019-01-02,0,0
2,20190103175451089289,0.0,2019-01-04,0,0
3,20190116130056566242,2.0,2019-01-17,2,2
4,20190117093459651115,0.0,2019-01-21,0,0


In [183]:
df_flag = pd.read_csv(os.path.join(file_path, 'df_flag.csv'))
df_flag['flag1'] = df_flag['late_days1'].apply(lambda x: 1 if x>=7 else 0)
df_flag['flag_max'] = df_flag['max_latedays'].apply(lambda x: 1 if x>=7 else 0)
df_flag['flag_sum'] = df_flag['sum_latedays'].apply(lambda x: 1 if x>=7 else 0)
remain_cols = ['order_no', 'effective_date','flag1', 'flag_max', 'flag_sum']
df_flag[remain_cols].to_csv(file_path + 'my_flag.csv', index = False)

In [ ]:
3)取上笔订单的表现和该笔订单的金额

In [20]:
query3 = '''
select t1.order_no as order_key,t1.approved_principal,t1.member_level,
t2.loan_principal,
t2.order_no,
t2.effective_date,
row_number()over(partition by t1.order_no order by t2.effective_date desc) as loan_rank
from dw_flb_approval_phl_approval_prod_approval_applications t1 
left join 
(select * from dw_flb_hx_phl_core_prod_core_loans
where product_name != 'CashNiJuan'  and loan_flow_status = 'LoanSuccess') t2 
on t1.customer_phone_num = t2.phone_number
where t1.product_name != 'CashNiJuan' 
and t1.order_type like 'Qua%'
and date(t1.apply_time) >='2019-04-01'
and date(t1.apply_time) <='2019-12-23'
and (date(t1.apply_time)> t2.effective_date or t2.effective_date is null)
'''

In [21]:
df3 = data_qry.database_gn(query3)
df3 = df3[df3['loan_rank'] == 1]
df3.info()
df3.head()
df3.to_csv(os.path.join(file_path, 'df3.csv'), index=False)

Successful connection
<class 'pandas.core.frame.DataFrame'>
Int64Index: 33338 entries, 0 to 67778
Data columns (total 7 columns):
order_key             33338 non-null object
approved_principal    33338 non-null object
member_level          33338 non-null int64
loan_principal        33338 non-null object
order_no              33338 non-null object
effective_date        33338 non-null object
loan_rank             33338 non-null int64
dtypes: int64(2), object(5)
memory usage: 2.0+ MB


,order_key,approved_principal,member_level,loan_principal,order_no,effective_date,loan_rank
0,Jp201910280149,3500.0,1,3000.0,Jp201909240233,2019-09-24,1
1,Jp201910280242,3500.0,1,2000.0,Jp201909230564,2019-09-24,1
2,Jp201910300455,3500.0,1,3000.0,Jp201909230527,2019-09-24,1
3,Jp201911050356,3500.0,1,3000.0,Jp201909230620,2019-09-24,1
4,Jp201911070644,3500.0,1,3000.0,Jp201909230398,2019-09-24,1


In [36]:
df1 = pd.read_csv(os.path.join(file_path, 'df.csv'))
df31 = pd.merge(df3, df1[['order_no', 'actual_paid_off_date','apply_time','late_days','extend_period']], on='order_no',how = 'left')
df31 = df31.drop_duplicates()

In [37]:
df31['repay_start_last'] = df31.apply(lambda x: (pd.to_datetime(x['actual_paid_off_date']) - pd.to_datetime(x['effective_date'])).days if x['actual_paid_off_date'] != '1970-01-01' else
                                             -9999,  axis =1)

In [38]:
my_group3 = df31.groupby('order_key')
frame31 = pd.DataFrame()
frame31['放款金额'] = my_group3['approved_principal'].min()
frame31['等级'] = my_group3['member_level'].min()
frame31['上笔放款金额'] = my_group3['loan_principal'].min()
frame31['上笔逾期天数'] = my_group3['late_days'].sum()
frame31['上笔最大逾期天数'] = my_group3['late_days'].max()
frame31['上笔最小逾期天数'] = my_group3['late_days'].min()
frame31['上笔订单结清天数'] = my_group3['repay_start_last'].min()
frame31['上笔订单展期次数'] = my_group3['extend_period'].max()
frame31.head()


,放款金额,等级,上笔放款金额,上笔逾期天数,上笔最大逾期天数,上笔最小逾期天数,上笔订单结清天数,上笔订单展期次数
order_key,,,,,,,,
Jp201910280149,3500.0,1,3000.0,0,0,0,14,0
Jp201910280242,3500.0,1,2000.0,0,0,0,8,0
Jp201910300455,3500.0,1,3000.0,0,0,0,13,0
Jp201911050356,3500.0,1,3000.0,0,0,0,13,0
Jp201911070644,3500.0,1,3000.0,1,1,1,14,0


In [39]:
my_var1 = pd.read_csv(file_path + 'my_var1.csv')
my_var2 = pd.merge(my_var1,frame31.reset_index(),on = 'order_key',how = 'left')
my_var2 = my_var2.drop(['第一次放款距今天数', '最近一次放款距今天数'],axis = 1)
my_var2.to_csv(file_path + 'my_var2.csv', index = False)

In [43]:
query4 = '''
select t1.apply_date, t1.order_type,t1.approved_principal,
t2.effective_date,
case when t2.effective_date is not null then 1 else 0 end as if_fund,
count(1)
from 
(select order_no, date(apply_time) as apply_date,order_type,approved_principal
from dw_flb_approval_phl_approval_prod_approval_applications 
where product_name != 'CashNiJuan' 
and  order_type like 'Qua%'
and date(apply_time) >='2019-04-01') t1 
left join 
(select order_no,effective_date
from dw_flb_hx_phl_core_prod_core_loans
where loan_flow_status = 'LoanSuccess') t2 
on t1.order_no = t2.order_no
group by t1.apply_date, t1.order_type,t2.effective_date,t1.approved_principal,
case when t2.effective_date is not null then 1 else 0 end ;
'''

In [45]:
df4 = data_qry.database_gn(query4)
df4.info()
df4.head()
df4.to_csv(file_path + 'df_anal.csv', index = False)

Successful connection
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2885 entries, 0 to 2884
Data columns (total 6 columns):
apply_date            2885 non-null object
order_type            2885 non-null object
approved_principal    2885 non-null object
effective_date        1919 non-null object
if_fund               2885 non-null int64
count                 2885 non-null int64
dtypes: int64(2), object(4)
memory usage: 135.4+ KB


,apply_date,order_type,approved_principal,effective_date,if_fund,count
0,2019-04-01,QualityApplicationOrder,3000.0,2019-04-01,1,23
1,2019-04-01,QualityApplicationOrder,3000.0,2019-04-02,1,31
2,2019-04-01,QualityApplicationOrder,3000.0,None,0,2
3,2019-04-01,QualityApplicationOrderClassB,2000.0,2019-04-03,1,1
4,2019-04-01,QualityApplicationOrderClassB,2000.0,None,0,1


In [55]:
df_anal = pd.read_csv(file_path + 'df_anal.csv')
my_summary = pd.ExcelWriter(file_path + 'my_summary1226.xlsx')
summary1 = pd.pivot_table(df_anal,index = 'apply_date', columns=['order_type', 'if_fund'], values='count', aggfunc='sum',fill_value=0,margins=True).sort_values(by='apply_date', ascending=False)
summary2 = pd.pivot_table(df_anal,index = 'apply_date', columns=['order_type', 'approved_principal'], values='count', aggfunc='sum',fill_value=0,margins=True).sort_values(by='apply_date', ascending=False)
summary3 = pd.pivot_table(df_anal,index = 'apply_date', columns=['approved_principal'], values='count', aggfunc='sum',fill_value=0,margins=True).sort_values(by='apply_date', ascending=False)
summary1.to_excel(my_summary, sheet_name = 'summary1')
summary2.to_excel(my_summary, sheet_name = 'summary2')
summary3.to_excel(my_summary, sheet_name = 'summary3')
my_summary.save()

In [8]:
query5 = '''
select t1.order_no,t1.apply_date, t1.order_type,t1.approved_principal,
t2.effective_date,
case when t2.effective_date is not null then 1 else 0 end as if_fund
from 
(select order_no, date(apply_time) as apply_date,order_type,approved_principal
from dw_flb_approval_phl_approval_prod_approval_applications 
where product_name != 'CashNiJuan' 
and  order_type like 'Qua%'
and date(apply_time) >='2019-04-01'
and date(apply_time) <='2019-12-23'
) t1 
left join 
(select order_no,effective_date
from dw_flb_hx_phl_core_prod_core_loans
where loan_flow_status = 'LoanSuccess') t2 
on t1.order_no = t2.order_no
'''

In [9]:
df5 = data_qry.database_gn(query5)
df5.info()
df5.head()
df5.to_csv(file_path + 'df_fund_flag.csv', index = False)

Successful connection
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33342 entries, 0 to 33341
Data columns (total 6 columns):
order_no              33342 non-null object
apply_date            33342 non-null object
order_type            33342 non-null object
approved_principal    33342 non-null object
effective_date        29408 non-null object
if_fund               33342 non-null int64
dtypes: int64(1), object(5)
memory usage: 1.5+ MB


,order_no,apply_date,order_type,approved_principal,effective_date,if_fund
0,P2g201908190199,2019-08-19,QualityApplicationOrder,2000.0,2019-08-19,1
1,P2g201905280060,2019-05-28,QualityApplicationOrder,3000.0,2019-05-29,1
2,P2g201905280074,2019-05-28,QualityApplicationOrder,3000.0,2019-05-28,1
3,P2g201905280115,2019-05-28,QualityApplicationOrder,3000.0,2019-05-28,1
4,P2g201905280148,2019-05-28,QualityApplicationOrder,3000.0,2019-05-28,1
